<a href="https://colab.research.google.com/github/chakrateja70/claude-Agent/blob/main/ClaudeAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://chatgpt.com/c/67a198f6-fd28-800c-9a08-0e0dd41e3ac4

# *Required Pacakges*

In [ ]:
!pip install langchain pinecone-client anthropic voyageai langchain-voyageai python-dotenv pypdf langchain-community


In [23]:
import os
from dotenv import load_dotenv

# Load API keys from environment variables
load_dotenv()

# Set API keys (replace with actual keys)
os.environ["PINECONE_API_KEY"] = "your-pinecone-api-key"
os.environ["ANTHROPIC_API_KEY"] = "your-claude-api-key"
os.environ["VOYAGE_API_KEY"] = "your-voyage-api-key"


In [25]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

def load_documents(folder_path: str) -> List[Document]:
    pages = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        pages.extend(loader.load())
    return pages

folder_path = "/content/"
pages = load_documents(folder_path)
print(f"Loaded {len(pages)} documents from the folder.")


Unsupported file type: .config
Unsupported file type: sample_data
Loaded 8 documents from the folder.
